In [2]:
import pyspark
from pathlib import Path
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")

24/08/05 19:33:17 WARN Utils: Your hostname, codespaces-0918c7 resolves to a loopback address: 127.0.0.1; using 10.0.1.86 instead (on interface eth0)
24/08/05 19:33:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/08/05 19:33:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data_folder = Path('../../data')
fhv = data_folder / Path("fhvhv_tripdata_2021-01.parquet")

!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

In [5]:
df = spark.read \
    .option("header", "true") \
    .parquet(str(fhv))

selected_columns = ['hvfhs_license_num',
     'dispatching_base_num',
    'pickup_datetime',
    'dropoff_datetime',
    'PULocationID',
     'DOLocationID',
    'shared_request_flag'
    ]

df = df[selected_columns]
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|shared_request_flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------------------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|                  N|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|                  N|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|                  N|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|                  N|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|     

In [6]:
df.dtypes

[('hvfhs_license_num', 'string'),
 ('dispatching_base_num', 'string'),
 ('pickup_datetime', 'timestamp'),
 ('dropoff_datetime', 'timestamp'),
 ('PULocationID', 'bigint'),
 ('DOLocationID', 'bigint'),
 ('shared_request_flag', 'string')]

In [7]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('shared_request_flag', StringType(), True)])

In [8]:
from pyspark.sql import types

In [9]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('shared_request_flag', types.StringType(), True)
])


In [12]:
df = spark.read \
    .option("inferSchema","false") \
    .parquet(str(fhv))
    # .schema(schema) \
df = df[selected_columns]

In [13]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('shared_request_flag', StringType(), True)])

In [19]:
df = df.repartition(24)

In [21]:
df.write.parquet(str(data_folder / Path('fhvhv/2021/1')), mode = 'overwrite')

In [22]:
df = spark.read.parquet(str(data_folder / Path('fhvhv/2021/1')))

In [23]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- shared_request_flag: string (nullable = true)



In [25]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')\
    .filter( df['hvfhs_license_num'] == 'HV0003').show(10)

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-14 13:43:59|2021-01-14 14:12:17|         198|         234|
|2021-01-01 03:13:23|2021-01-01 04:04:00|          65|         265|
|2021-01-11 18:38:11|2021-01-11 19:10:54|         258|          52|
|2021-01-27 18:03:59|2021-01-27 18:11:14|         116|         244|
|2021-01-31 08:21:46|2021-01-31 08:30:52|         241|          18|
|2021-01-28 11:48:42|2021-01-28 12:02:24|         196|          82|
|2021-01-30 14:46:53|2021-01-30 14:59:23|         227|          14|
|2021-01-28 12:06:37|2021-01-28 12:19:05|         174|          18|
|2021-01-30 05:37:52|2021-01-30 05:48:28|         205|         130|
|2021-01-01 05:05:59|2021-01-01 05:24:16|          53|           7|
+-------------------+-------------------+------------+------------+
only showing top 10 rows



In [26]:
from pyspark.sql import functions as F

In [28]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime))  \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime))  \
    .show(10)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------------------+-----------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|shared_request_flag|pickup_date|dropoff_date|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------------------+-----------+------------+
|           HV0003|              B02764|2021-01-14 13:43:59|2021-01-14 14:12:17|         198|         234|                  N| 2021-01-14|  2021-01-14|
|           HV0003|              B02764|2021-01-01 03:13:23|2021-01-01 04:04:00|          65|         265|                  N| 2021-01-01|  2021-01-01|
|           HV0003|              B02682|2021-01-11 18:38:11|2021-01-11 19:10:54|         258|          52|                  N| 2021-01-11|  2021-01-11|
|           HV0005|              B02510|2021-01-26 08:19:47|2021-01-26 08:32:23|        

In [42]:
def sample_func(base_num):
    num = int (base_num[2:])
    if num % 7 == 0:
        return f's/{num:03x}'
    else:
        return f'e/{num:03x}'
        

In [43]:
sample_func('HV01232')

's/4d0'

In [44]:
sample_func_udf = F.udf(sample_func, returnType=types.StringType())

In [45]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime))  \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime))  \
    .withColumn('base_id', sample_func_udf(df.dispatching_base_num))  \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID', 'base_id') \
    .show(10)

+-----------+------------+------------+------------+-------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|base_id|
+-----------+------------+------------+------------+-------+
| 2021-01-14|  2021-01-14|         198|         234|  e/acc|
| 2021-01-01|  2021-01-01|          65|         265|  e/acc|
| 2021-01-11|  2021-01-11|         258|          52|  e/a7a|
| 2021-01-26|  2021-01-26|         239|         244|  e/9ce|
| 2021-01-27|  2021-01-27|         116|         244|  e/b3b|
| 2021-01-31|  2021-01-31|         241|          18|  e/acc|
| 2021-01-28|  2021-01-28|         196|          82|  e/b3e|
| 2021-01-03|  2021-01-03|         261|           1|  e/9ce|
| 2021-01-30|  2021-01-30|         227|          14|  e/b3c|
| 2021-01-28|  2021-01-28|         174|          18|  e/b37|
+-----------+------------+------------+------------+-------+
only showing top 10 rows

